# Results

In [1]:
from os import makedirs
from os.path import join

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from matplotlib.ticker import MaxNLocator

plt.rcParams.update({
    'text.latex.preamble': r'\usepackage{cmbright}',
    'text.usetex': True,
    'font.family': 'serif',
})
color = sns.color_palette('deep')

results_dir = '../rpaper'
plots_dir = 'paper_plots'

makedirs(plots_dir, exist_ok=True)

### Arch

In [ ]:
# def load_arch_df():
#     df_extra = pd.DataFrame(
#         [
#             ['mobilenetv3-smHM-075', 'MobileNetV3-SmHM',  1.02,  0.12],
#             ['mobilevitv2-050',       'MobileViTv2-0.5',    1.11,  1.05],
#             ['mobilenetv3-large-100', 'MobileNetV3-Large',  4.20,  0.63],
#             ['convnext-atto',         'ConvNeXtV1-Atto',    3.37,  1.62],
#             ['convnextv2-atto',       'ConvNeXtV2-Atto',    3.39,  1.62],
#             ['mobilevitv2-100',       'MobileViTv2-1.0',    4.39,  4.08],

#             ['densenet121',           'Densenet121',        6.95,  8.33],
#             ['mobilevitv2-200',       'MobileViTv2-2.0',   17.42, 16.11],
#             ['convnextv2-nano',       'ConvNeXtV2-Nano',   14.98,  7.21],
#             ['densenet161',           'Densenet161',       26.47, 22.70],
#             ['convnext-tiny',         'ConvNeXtV1-Tiny',   27.82, 28.60],
#             ['convnextv2-tiny',       'ConvNeXtV2-Tiny',   27.87, 28.60],
#         ],
#         columns=['run', 'Arch', 'Params', 'MACs']
#     ).set_index('run')

#     df = pd.read_csv(join(results_dir, 'arch_batch-size-64', 'exp_mtst.csv'))
#     df = df.iloc[:, :4]
#     df = df.rename(columns={'unseen': 'Unseen', 'seen': 'Seen', 'combined': 'HM'})
#     df['Unseen'] = df['Unseen'].str[:5].astype(float)
#     df['Seen'] = df['Seen'].str[:5].astype(float)
#     df['HM'] = df['HM'].str[:5].astype(float)
#     df = df.set_index('run')

#     df = pd.concat([df_extra, df], axis=1, join='inner')
#     df = df[['Arch', 'Unseen', 'Seen', 'HM', 'Params', 'MACs']]
#     df = df.reset_index(drop=True)
#     return df

# arch_df = load_arch_df()
# arch_df

In [ ]:
# def generate_arch_plot(df):
#     fig, (ax0, ax1) = plt.subplots(1, 2, figsize=(10, 5))

#     _color = color[:len(df)]

#     df.plot.scatter('Params', 'HM',
#                     ax=ax0, c=_color)
#     df.plot.scatter('MACs', 'HM',
#                     ax=ax1, c=_color)

#     for x, y, arch in zip(df['Params'], df['HM'], df['Arch']):
#         ax0.text(x, y, f'{arch}', ha='left', va='bottom')
#     xmin, xmax = ax0.get_xlim()
#     ax0.set_xlim(xmin, xmax + 0.35 * (xmax - xmin))
#     ax0.tick_params(axis='both', which='major', labelsize='small')

#     for x, y, arch in zip(df['MACs'], df['HM'], df['Arch']):
#         ax1.text(x, y, f'{arch}',
#                 ha='left', va='bottom')
#     xmin, xmax = ax1.get_xlim()
#     ax1.set_xlim(xmin, xmax + 0.35 * (xmax - xmin))
#     ax1.tick_params(axis='both', which='major', labelsize='small')

#     ax0.set_xlabel('Params (M)')
#     ax1.set_xlabel('MACs (G)')
#     ax1.set_yticklabels([])
#     ax1.yaxis.label.set_visible(False)

#     # plt.xticks(fontsize='small')
#     # plt.yticks(fontsize='small')

#     plt.tight_layout()
#     plt.savefig(join(plots_dir, "params-macs.pdf"))
#     plt.show()

# generate_arch_plot(arch_df)

### Resolution

### From Generalized FSL to Standard FSL

In [ ]:
results_dir = '../../Meta-CXR-dev-run/rpaper'

def load_gfsl_df(

    ):
    df = pd.read_csv(join(results_dir, 'gfsl', 'exp_mtst.csv'))

    df[['n-way', 'n-unseen', 'k-shot']] = df['run'].str.split('_', expand=True)
    df['n-way'] = df['n-way'].str[5:].astype(int)
    df['n-unseen'] = df['n-unseen'].str[7:].astype(int)
    df['k-shot'] = df['k-shot'].str[6:].astype(int)
    df['Unseen'] = df['unseen'].str[:5].astype(float)
    df['Seen'] = df['seen'].str[:5].astype(float)
    df['HM'] = df['hm'].str[:5].astype(float)
    df = df[['n-way', 'n-unseen', 'k-shot', 'Unseen', 'Seen', 'HM']]
    return df

gfsl_df = load_gfsl_df()
with open('gfsl.md', 'w') as f:
    f.write(gfsl_df.to_markdown(index=False) + '\n')
gfsl_df

In [ ]:
def generate_gfsl_df(df, k_shot):
    sns.set_palette('deep')
    fig, ax = plt.subplots(figsize=(6, 5))

    for n_way, c in zip((3, 4, 5), color):
        n_way_label = f'{n_way}-way'
        subdf = df[df['n-way'] == n_way]
        # subdf = subdf.rename(columns={'HM': n_way_label})
        # subdf.plot(x='n-unseen', y=n_way_label,
        #            style='o-', ax=ax, color=c)
        # subdf.plot(x='n-unseen', y='Unseen',
        #            style='s-', ax=ax, color=c, alpha=0.5)
        # subdf.plot(x='n-unseen', y='Seen',
        #            style='D-', ax=ax, color=c, alpha=0.5)

        subdf.plot(x='n-unseen', y='HM',
                   ax=ax,
                   marker='.', linestyle='solid',
                   color=c)
        subdf.plot(x='n-unseen', y='Unseen',
                   ax=ax,
                   marker='.', linestyle='dotted',
                   color=c, alpha=0.5)
        subdf.plot(x='n-unseen', y='Seen',
                   ax=ax,
                   marker='.', linestyle='dashed',
                   color=c, alpha=0.5)

        ax.get_legend().remove()

        from matplotlib.legend import Legend
        leg = Legend(ax, ax.get_lines()[-3:],
                     ['HM', 'Unseen', 'Seen'],
                     title=f'{n_way}-way',
                     fontsize='x-small',
                     edgecolor=c,
                     loc='upper right')
        bb = leg.get_bbox_to_anchor().transformed(ax.transAxes.inverted())
        y_offset = (3 - n_way) * 0.18
        bb.y0 += y_offset
        bb.y1 += y_offset
        leg.set_bbox_to_anchor(bb, transform = ax.transAxes)
        ax.add_artist(leg)

    ax.set_xticks(range(1, 6))
    ax.tick_params(axis='both', which='major', labelsize='small')
    ax.set_xlabel(r'$n$-unseen: Number of Unseen Pathologies')
    ax.set_ylabel('AUC')

    plt.title(f'From Generalized to Standard FSL (At least {k_shot}-shot)')

    plt.tight_layout()
    plt.savefig(join(plots_dir, "gfsl.pdf"))
    plt.show()


for k_shot, k_shot_df in gfsl_df.groupby('k-shot'):
    k_shot_df = k_shot_df.drop(columns=['k-shot'])
    generate_gfsl_df(k_shot_df, k_shot)
    print(k_shot_df)



# for k_shot in gfsl_df['k-shot'].unique():
#     print(k_shot)
    #generate_gfsl_df(gfsl_df)